# Pre-trained model using MobileNetV2

## Setup and imports

In [1]:
# import pre-trained model MobileNetV2
from keras.applications import MobileNetV2
from keras import layers, models, utils
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import requests
import time
import pandas as pd
from urllib.parse import quote
from selenium import webdriver
import sys
import numpy as np


In [2]:
# Add a directory path to sys.path
sys.path.append('/Users/Jason/Code/Yassinoko/dynamic-players-insights/playerprediction/')


In [3]:
os.getcwd()


'/Users/niknmirosh/code/Yassinoko/dynamic-players-insights/notebooks'

In [4]:
!pwd


/Users/niknmirosh/code/Yassinoko/dynamic-players-insights/notebooks


In [7]:
# import get_player_images function
from ml_logic.get_images import get_players_images
from ml_logic.class_preprocessing import preprocessing_classes
from ml_logic.image_preprocessing import processing_images


ModuleNotFoundError: No module named 'ml_logic'

## Load images into notebook

In [ ]:
player_images = get_players_images()


In [ ]:
player_images.keys()


In [ ]:
# create dataframe
df = pd.DataFrame(player_images)
df.shape # 1158 players with image and name as columns


## Image preprocessing


In [9]:
# preprocessing the images and labels
X, y = processing_images(player_images)
# encoding the labels
y = preprocessing_classes(y)


100%|██████████| 1158/1158 [00:00<00:00, 1173.18it/s]


In [10]:
X.shape, y.shape


((1158, 408, 612, 3), (1158, 44))

## Test and Train split with shuffle while retaining all classes in test and train

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True) # stratify does not work on classes that have only one instance


In [12]:
X_train.shape, y_train.shape


((810, 408, 612, 3), (810, 44))

## Model building

In [13]:
# define the model

def load_model():
    model = MobileNetV2(
        input_shape=X_train.shape[1:],
        alpha=1.0,
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        pooling=None,
        classes=y_train.shape[-1],
        classifier_activation="softmax",
        )
    return model


In [29]:
# set the first layers to be non-trainable
def set_nontrainable_layers(model):
    model.trainable = False
    return model


In [16]:
# add trainable layers

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(y_train.shape[-1], activation='softmax')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    return model


In [17]:
model = load_model()
model = add_last_layers(model)


In [18]:
# compile the model
model.compile(loss= 'categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'],
                 )


In [19]:
# Define early stopping
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=2, restore_best_weights=True)

# fit model
history = model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    epochs = 5,
    batch_size = 32,
    verbose = 1,
    callbacks = [es],
    )


Epoch 1/5


2023-11-23 15:59:30.583602: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


21/21 [==============================] - 143s 6s/step - loss: 26.5808 - accuracy: 0.0340 - val_loss: 4748.3545 - val_accuracy: 0.0247
Epoch 2/5
21/21 [==============================] - 136s 6s/step - loss: 3.8611 - accuracy: 0.0478 - val_loss: 5471.4790 - val_accuracy: 0.0247
Epoch 3/5
21/21 [==============================] - ETA: 0s - loss: 3.7185 - accuracy: 0.0586

KeyboardInterrupt: 

In [44]:
model.evaluate(X_test, y_test)


11/11 [==============================] - 8s 731ms/step - loss: 7.6094 - accuracy: 0.1724


[7.609419345855713, 0.17241379618644714]

In [41]:
test_accuracy = np.mean(history.history['accuracy'])
print(f"test_accuracy = {round(test_accuracy,2)*100} %")


test_accuracy = 90.0 %
